In [0]:
sas="<sas token of Blob storage>"
dbutils.fs.mount(
  source = "wasbs://<container name>@<storage account name>.blob.core.windows.net",
  mount_point = "/mnt/data"#<put your preferred name of file instead of "data">
  extra_configs = {"fs.azure.sas. <container name>.<storage account name>.blob.core.windows.net":sas})


In [0]:
from datetime import date
import pandas as pd

# The github repo gets updated every Monday. So we fetch current date and plug it in the url. Assuming we schedule the notebook to run every Monday
# The name of the files in the Github repo is in a standard format- eg: "2021-02-01-COVIDhub-ensemble.csv" So we can dynamically create the url every monday and request for the csv file.
today = '2021-01-18'  
#today = date.today()
# If you schedule the notebook to run every Monday then comment the 6th line and run the 7th line
#print(str(today))
url_part = 'https://raw.githubusercontent.com/reichlab/covid19-forecast-hub/master/data-processed/COVIDhub-ensemble/'+str(today)+'-COVIDhub-ensemble.csv'

df = pd.read_csv(url_part,header=0)
df = df[df.location != 'US'] #Remove rows containing US as location

locations=pd.read_csv('/dbfs/mnt/data/locations.csv') # Contains location name and location code. Will be sued to map location code to location
# Prediction data from github only contains location code. So using this csv file we will add location name as another column by mapping it to location code
# Take a look at their github repo- https://github.com/reichlab/covid19-forecast-hub/tree/master/data-processed/COVIDhub-ensemble

locations = locations[locations.location != 'US'] #Remove rows containing US as location
locations['location']=locations['location'].astype(int) 

deaths_states_loc=pd.read_csv('/dbfs/mnt/data/deaths_states_loc.csv')
# The prediction data contains predictions for all states and counties in the same file.
# The variable deaths_states_loc contain the list of states so that we can store the predictions for states in a separate file.



2021-01-15

--------------------------------------------------------------------------- 
 HTTPError Traceback (most recent call last)
 <command-1387113928693019> in <module> 
 7 url_part = 'https://raw.githubusercontent.com/reichlab/covid19-forecast-hub/master/data-processed/COVIDhub-ensemble/' + str ( today ) + '-COVIDhub-ensemble.csv' 
 8 
 ----> 9 df = pd . read_csv ( url_part , header = 0 ) 
 10 df = df [ df . location != 'US' ] 
 11 

 /databricks/python/lib/python3.7/site-packages/pandas/io/parsers.py in parser_f (filepath_or_buffer, sep, delimiter, header, names, index_col, usecols, squeeze, prefix, mangle_dupe_cols, dtype, engine, converters, true_values, false_values, skipinitialspace, skiprows, skipfooter, nrows, na_values, keep_default_na, na_filter, verbose, skip_blank_lines, parse_dates, infer_datetime_format, keep_date_col, date_parser, dayfirst, cache_dates, iterator, chunksize, compression, thousands, decimal, lineterminator, quotechar, quoting, doublequote, escapechar, comment, encoding, dialect, error_bad_lines, warn_bad_lines, delim_whitespace, low_memory, memory_map, float_precision) 
 674 )
 675 
 --> 676 return _read ( filepath_or_buffer , kwds ) 
 677 
 678 parser_f . __name__ = name

 /databricks/python/lib/python3.7/site-packages/pandas/io/parsers.py in _read (filepath_or_buffer, kwds) 
 429 # See https://github.com/python/mypy/issues/1297 
 430 fp_or_buf, _, compression, should_close = get_filepath_or_buffer(
 --> 431 filepath_or_buffer , encoding , compression
 432 )
 433 kwds [ "compression" ] = compression

 /databricks/python/lib/python3.7/site-packages/pandas/io/common.py in get_filepath_or_buffer (filepath_or_buffer, encoding, compression, mode) 
 170 
 171 if isinstance ( filepath_or_buffer , str ) and is_url ( filepath_or_buffer ) : 
 --> 172 req = urlopen ( filepath_or_buffer ) 
 173 content_encoding = req . headers . get ( "Content-Encoding" , None ) 
 174 if content_encoding == "gzip" : 

 /databricks/python/lib/python3.7/site-packages/pandas/io/common.py in urlopen (*args, **kwargs) 
 139 import urllib . request
 140 
 --> 141 return urllib . request . urlopen ( * args , ** kwargs ) 
 142 
 143 

 /usr/lib/python3.7/urllib/request.py in urlopen (url, data, timeout, cafile, capath, cadefault, context) 
 220 else : 
 221 opener = _opener
 --> 222 return opener . open ( url , data , timeout ) 
 223 
 224 def install_opener ( opener ) : 

 /usr/lib/python3.7/urllib/request.py in open (self, fullurl, data, timeout) 
 529 for processor in self . process_response . get ( protocol , [ ] ) : 
 530 meth = getattr ( processor , meth_name ) 
 --> 531 response = meth ( req , response ) 
 532 
 533 return response

 /usr/lib/python3.7/urllib/request.py in http_response (self, request, response) 
 639 if not ( 200 <= code < 300 ) : 
 640 response = self.parent.error(
 --> 641 'http', request, response, code, msg, hdrs)
 642 
 643 return response

 /usr/lib/python3.7/urllib/request.py in error (self, proto, *args) 
 567 if http_err : 
 568 args = ( dict , 'default' , 'http_error_default' ) + orig_args
 --> 569 return self . _call_chain ( * args ) 
 570 
 571 # XXX probably also want an abstract factory that knows when it makes 

 /usr/lib/python3.7/urllib/request.py in _call_chain (self, chain, kind, meth_name, *args) 
 501 for handler in handlers : 
 502 func = getattr ( handler , meth_name ) 
 --> 503 result = func ( * args ) 
 504 if result is not None : 
 505 return result

 /usr/lib/python3.7/urllib/request.py in http_error_default (self, req, fp, code, msg, hdrs) 
 647 class HTTPDefaultErrorHandler ( BaseHandler ) : 
 648 def http_error_default ( self , req , fp , code , msg , hdrs ) : 
 --> 649 raise HTTPError ( req . full_url , code , msg , hdrs , fp ) 
 650 
 651 class HTTPRedirectHandler ( BaseHandler ) : 

 HTTPError : HTTP Error 404: Not Found

In [0]:
# We need predictions for the next 1 week and we need the point values for the predictions 
# So we filter out rows with target value of "1 wk ahead inc death","1 wk ahead inc case" and type column as "point"

df_d=df[(df.target=="1 wk ahead inc death")&(df.type=="point")] 
df_d['location']=df_d['location'].astype(int)
df_c=df[((df.target=="1 wk ahead inc case")&(df.type=="point"))]
df_c['location']=df_c['location'].astype(int)
# Save cases and deaths predictions in separate variables df_c and df_d respectively

# Add the location name cloumn from the location.csv by mapping it with location code column

df_c=pd.merge(df_c,locations,on='location',how='left')
df_d=pd.merge(df_d,locations,on='location',how='left')

df_c = df_c.drop(columns=['type', 'quantile', 'population','abbreviation','location','target'])
df_d = df_d.drop(columns=['type', 'quantile', 'population','abbreviation','location','target'])

/local_disk0/tmp/1610703946271-0/PythonShell.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
 # The first line is intentionally left blank because warnings.warn in a notebook will include it.
/local_disk0/tmp/1610703946271-0/PythonShell.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
 # Databricks Python Shell

In [0]:
death_states_list=deaths_states_loc['Deaths_state'].tolist()

Out[29]: ['Guam',
 'Northern Mariana Islands',
 'Puerto Rico',
 'Virgin Islands',
 'District of Columbia',
 'Alabama',
 'Alaska',
 'Arizona',
 'Arkansas',
 'California',
 'Colorado',
 'Connecticut',
 'Delaware',
 'Florida',
 'Hawaii',
 'Idaho',
 'Iowa',
 'Kansas',
 'Louisiana',
 'Maine',
 'Maryland',
 'Massachusetts',
 'Mississippi',
 'Missouri',
 'Montana',
 'Nebraska',
 'Nevada',
 'New Hampshire',
 'New Jersey',
 'New Mexico',
 'New York',
 'North Carolina',
 'North Dakota',
 'Oklahoma',
 'Oregon',
 'Pennsylvania',
 'Rhode Island',
 'South Carolina',
 'South Dakota',
 'Tennessee',
 'Texas',
 'Utah',
 'Vermont',
 'Virginia',
 'Washington',
 'West Virginia',
 'Wyoming',
 'Illinois',
 'Indiana',
 'Kentucky',
 'Michigan',
 'Minnesota',
 'Ohio',
 'Wisconsin',
 'Georgia',
 'American Samoa']

In [0]:
# Deaths are only predicted for states and not for counties
# So we remove the states predictions from county predictions variable df_c

county_cases=df_c[~df_c['location_name'].isin(death_states_list)]
county_cases.columns = ['forecast_date', 'target_end_date', 'predicted_cases_1wk', 'location_name']
county_cases=county_cases.sort_values(by=['location_name','forecast_date'])
county_cases = county_cases.reset_index(drop=True).set_index('forecast_date')

Out[31]:

,target_end_date,predicted_cases_1wk,location_name
forecast_date,,,
2021-01-11,2021-01-16,79,Abbeville County
2021-01-11,2021-01-16,296,Acadia County
2021-01-11,2021-01-16,174,Accomack County
2021-01-11,2021-01-16,1615,Ada County
2021-01-11,2021-01-16,33,Adair County
...,...,...,...
2021-01-11,2021-01-16,29,Yuma County
2021-01-11,2021-01-16,2142,Yuma County
2021-01-11,2021-01-16,144,Zapata County


In [0]:
# Deaths are only predicted for states and not for counties
# The variable df_d contains only the death predictions for states. The cases predictions for states are in the variable df_c
# The variable df_c contains cases predictions for states as well as counties. So we merge both these variables to get cases and deaths predictions for states.

Key_Column=['forecast_date','location_name','target_end_date']
states_cases_deaths = pd.merge(df_d, df_c,  how='left', left_on=Key_Column, right_on = Key_Column)
states_cases_deaths.columns = ['forecast_date', 'target_end_date', 'predicted_deaths_1wk', 'location_name', 'predicted_cases_1wk']
states_cases_deaths = states_cases_deaths[['forecast_date', 'target_end_date', 'predicted_deaths_1wk','predicted_cases_1wk','location_name']]
states_cases_deaths=states_cases_deaths.sort_values(by=['location_name','forecast_date'])
states_cases_deaths = states_cases_deaths.reset_index(drop=True).set_index('forecast_date')

Out[33]:

,target_end_date,predicted_deaths_1wk,predicted_cases_1wk,location_name
forecast_date,,,,
2021-01-11,2021-01-16,348,30103,Alabama
2021-01-11,2021-01-16,12,2160,Alaska
2021-01-11,2021-01-16,0,0,American Samoa
2021-01-11,2021-01-16,882,66029,Arizona
2021-01-11,2021-01-16,281,20396,Arkansas
2021-01-11,2021-01-16,2876,283648,California
2021-01-11,2021-01-16,292,18635,Colorado
2021-01-11,2021-01-16,233,15979,Connecticut
2021-01-11,2021-01-16,40,5571,Delaware


In [0]:
#We append the new predictions with the historical prediction if the historical predictions are present in your storage.
#Else we create a new file
try{
county_cases_hist=pd.read_csv('/dbfs/mnt/data/county_cases.csv')
states_cases_deaths_hist=pd.read_csv('/dbfs/mnt/data/states_cases_deaths.csv')

county_cases = pd.concat([county_cases_hist, county_cases])
states_cases= pd.concat([states_cases_deaths_hist, states_cases_deaths])
}
except{
  county_cases = county_cases
  states_cases= states_cases_deaths 
}

In [0]:
#Wirte the updated cases and deaths predition file back to blob storage
output_csv = '/dbfs/mnt/data/county_cases.csv'
dbutils.fs.put(output_csv, "", overwrite=True)
county_cases.to_csv(output_csv)

output_csv = '/dbfs/mnt/data/states_cases_deaths.csv'
dbutils.fs.put(output_csv, "", overwrite=True)
states_cases.to_csv(output_csv)
